# Open the Black-box using model-agnostic Explanation

In [1]:
from IPython.display import display, Markdown

import numpy as np
import lime
import lime.lime_tabular
import pandas as pd
import sklearn
import sklearn.datasets
import sklearn.ensemble

np.random.seed(1)
pd.options.mode.chained_assignment = None # hide some warnings

## Import the dataset

In [2]:
vat_data = pd.read_csv('dataset_clean_correct_v02.csv', delimiter=";")
if vat_data.isnull().values.any(): vat_data = vat_data.dropna()
vat_data_inputs = vat_data[['Ship_from','Ship_to','Incoterm','Buyer','VAT_ID','Material_Tax']]
vat_data_target = vat_data['Taxcode']

vat_data_inputs["Material_Tax"] = vat_data_inputs["Material_Tax"].astype('category')
vat_data_inputs["Incoterm"] = vat_data_inputs["Incoterm"].astype('category')

## EDA

In [3]:
pd.concat([vat_data_inputs, vat_data_target], axis=1).head()

Ship_from Ship_to Incoterm Buyer VAT_ID Material_Tax            Taxcode
0        AT      AT        0    AT     AT            1  domestic full tax
1        AT      AT        0    NL     AT            1  domestic full tax
2        AT      ES        0    CH     EU            1    intra-EU supply
3        AT      DE        1    AT     EU            1  domestic full tax
4        AT      AT        0    AT     AT            1  domestic full tax

**Ship_from**: origin country of the shipment  
**Ship_to**: destination country of the shipment   
**Incoterms**: who bears the risk legally (0 - seller, 1 - shipto party)  
**Buyer**: country of buyer  
**VAT_ID**: VAT_ID of shipto party  
**Material_Tax**: if shipment is taxable in Austria (0 - taxable, 1 - taxable)  
**Taxcode**: how shipment is taxed  

In [4]:
pd.concat([vat_data_inputs, vat_data_target], axis=1).describe()

Ship_from Ship_to  Incoterm  Buyer VAT_ID  Material_Tax  \
count      10000   10000     10000  10000  10000         10000   
unique         1       7         2      7      3             2   
top           AT      AT         0     AT     AT             1   
freq       10000    4031      7493   3987   4815          8058   

                  Taxcode  
count               10000  
unique                  4  
top     domestic full tax  
freq                 4572

### Display all rules in tabular form (148 possible combinations)

In [5]:
vat_data[['Ship_from', 'Ship_to','Buyer','VAT_ID','Material_Tax','Incoterm','Taxcode']].drop_duplicates()

Ship_from Ship_to Buyer        VAT_ID  Material_Tax  Incoterm  \
0           AT      AT    AT            AT             1         0   
1           AT      AT    NL            AT             1         0   
2           AT      ES    CH            EU             1         0   
3           AT      DE    AT            EU             1         1   
5           AT      RU    AT            AT             1         1   
6           AT      DE    RU            EU             1         0   
7           AT      NL    RU            EU             1         1   
8           AT      NL    RU            EU             1         0   
11          AT      AT    ES            AT             1         1   
12          AT      NL    AT            EU             1         0   
13          AT      AT    CH            AT             1         0   
14          AT      CH    CH  not required             1         0   
15          AT      IT    CH            EU             1         0   
16          AT      CH    NL  not required             1         0   
17          AT      DE    ES            EU             1         0   
18          AT      AT    IT            AT             0         1   
19          AT      CH    AT            AT             1         0   
20          AT      DE    AT            EU             1         0   
21          AT      AT    AT            AT             0         0   
22          AT      NL    ES            EU             0         1   
23          AT      AT    CH            AT             1         1   
24          AT      AT    IT            AT             1         1   
25          AT      AT    NL            AT             0         0   
28          AT      RU    DE  not required             1         0   
30          AT      IT    AT            EU             1         0   
31          AT      ES    DE            EU             1         0   
33          AT      AT    RU            AT             1         1   
34          AT      RU    AT            AT             1         0   
35          AT      NL    ES            EU             1         1   
38          AT      CH    AT            AT             0         0   
...        ...     ...   ...           ...           ...       ...   
1175        AT      AT    ES            AT             0         1   
1195        AT      DE    DE            EU             0         1   
1201        AT      CH    RU  not required             0         1   
1303        AT      DE    DE            EU             1         1   
1309        AT      IT    IT            EU             1         1   
1313        AT      NL    RU            EU             0         1   
1319        AT      IT    RU            EU             0         1   
1338        AT      DE    IT            EU             1         1   
1369        AT      ES    CH            EU             0         0   
1493        AT      NL    ES            EU             0         0   
1497        AT      RU    CH  not required             0         0   
1549        AT      NL    IT            EU             0         0   
1573        AT      RU    RU  not required             0         1   
1575        AT      DE    CH            EU             0         1   
1635        AT      ES    DE            EU             0         0   
1664        AT      RU    ES  not required             0         1   
1681        AT      NL    NL            EU             0         1   
1750        AT      ES    RU            EU             0         1   
1761        AT      IT    CH            EU             0         1   
1936        AT      CH    DE  not required             0         1   
2038        AT      DE    RU            EU             0         0   
2172        AT      RU    CH  not required             0         1   
2627        AT      IT    NL            EU             0         1   
2917        AT      IT    IT            EU             0         1   
3289        AT      DE    NL            EU             0         1   
3376        AT 

### Visualize Rules (R ggparallel)

<img src='hammock_plot.png'></img>

## Preprocessing

### Label Encoder for categorical inputs

In [6]:
encoders=dict()
encoded_vat_data_inputs = dict()
cat_names = {} # keep track of variable levels

for idx, col in enumerate(vat_data_inputs.columns):
    encoders[col] = sklearn.preprocessing.LabelEncoder()

    encoders[col].fit(vat_data_inputs[col].values.astype('str'))
    encoded_vat_data_inputs[col] = encoders[col].transform(vat_data_inputs[col].values.astype('str'))
    cat_names[idx] = list(encoders[col].classes_)

encoded_vat_data_inputs = pd.DataFrame(encoded_vat_data_inputs)
encoded_vat_data_inputs = encoded_vat_data_inputs[vat_data_inputs.columns] # keep original order

display(Markdown('**Dictionary of Categories of Columns**'))
print(cat_names)
print('\n')
display(Markdown('**Label-encoded input data**'))
display(encoded_vat_data_inputs.head())

**Dictionary of Categories of Columns**

{0: ['AT'], 1: ['AT', 'CH', 'DE', 'ES', 'IT', 'NL', 'RU'], 2: ['0', '1'], 3: ['AT', 'CH', 'DE', 'ES', 'IT', 'NL', 'RU'], 4: ['AT', 'EU', 'not required'], 5: ['0', '1']}




**Label-encoded input data**

Ship_from  Ship_to  Incoterm  Buyer  VAT_ID  Material_Tax
0          0        0         0      0       0             1
1          0        0         0      5       0             1
2          0        3         0      1       1             1
3          0        2         1      0       1             1
4          0        0         0      0       0             1

In [7]:
target_encoder = sklearn.preprocessing.LabelEncoder()
target_encoder.fit(vat_data_target.values)
encoded_vat_data_target = target_encoder.transform(vat_data_target.values)
encoded_vat_data_target = pd.DataFrame(encoded_vat_data_target,columns=['Taxcode'])

In [8]:
class_names = list(vat_data_target.astype('category').cat.categories)

In [9]:
display(Markdown('**List of Target Values**'))
display(class_names)
print('\n')
display(Markdown('**Label-encoded target column**'))
display(encoded_vat_data_target.head())

**List of Target Values**

['domestic full tax', 'domestic no tax', 'export', 'intra-EU supply']

**Label-encoded target column**

Taxcode
0        0
1        0
2        3
3        0
4        0

### Splitting the dataset into training and testing sets

In [17]:
train, test, labels_train, labels_test = sklearn.model_selection.train_test_split(encoded_vat_data_inputs.values, encoded_vat_data_target.values, train_size=0.80, test_size=0.20)

### One-Hot-Encoding

Turning label-encoded integer columns into 1 - 0 columns for each possible value.

In [11]:
ohe = sklearn.preprocessing.OneHotEncoder(sparse=False)
train_ohe = ohe.fit_transform(train)
test_ohe = ohe.transform(test)
display(pd.DataFrame(train_ohe[:5]))

c:\users\luvi\documents\winpython-64bit-3.6.2.0zero\python-3.6.2.amd64\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


0    1    2    3    4    5    6    7    8    9  ...    12   13   14   15  \
0  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0 ...   0.0  0.0  0.0  0.0   
1  1.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0 ...   0.0  1.0  0.0  0.0   
2  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  1.0  0.0 ...   0.0  0.0  0.0  0.0   
3  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0 ...   0.0  1.0  0.0  0.0   
4  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0 ...   0.0  1.0  0.0  0.0   

    16   17   18   19   20   21  
0  0.0  1.0  0.0  0.0  0.0  1.0  
1  0.0  0.0  1.0  0.0  1.0  0.0  
2  0.0  0.0  1.0  0.0  0.0  1.0  
3  0.0  0.0  0.0  1.0  0.0  1.0  
4  0.0  0.0  1.0  0.0  0.0  1.0  

[5 rows x 22 columns]

## Random Forest

In [12]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth=5, # maximum number of splits
                           n_estimators=10, # number of trees
                           max_features=1) # maximum number of features considered for each split

## Lime Explanation object

In [13]:
#create the explainer
explainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names = list(vat_data_inputs.columns),
                                                   categorical_features = list(np.arange(len(vat_data_inputs.columns))),
                                                   categorical_names = cat_names,
                                                   class_names=class_names,
                                                   discretize_continuous=True)

c:\users\luvi\documents\winpython-64bit-3.6.2.0zero\python-3.6.2.amd64\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


## Train Model

In [14]:
rf.fit(train_ohe, labels_train.ravel())
    
score = rf.score(test_ohe, labels_test.ravel())

display(Markdown('**Random Forest Model**'))
print("accuracy: ","%12.2f%%" % (score*100.0))

display(Markdown("**parameters:**"))
rf.get_params()


**Random Forest Model**

accuracy:         97.35%


**parameters:**

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': 1,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

## Select observation to inspect

In [15]:
import ipywidgets as widgets

In [36]:
ship_from = widgets.Dropdown(
    options=vat_data_inputs.Ship_from.unique(),
    value='AT',
    description='Ship_from:',
    disabled=False,
)
ship_to = widgets.Dropdown(
    options=vat_data_inputs.Ship_to.unique(),
    value='AT',
    description='Ship_to:',
    disabled=False,
)
incoterm = widgets.Dropdown(
    options=vat_data_inputs.Incoterm.unique(),
    value=0,
    description='Incoterm:',
    disabled=False,
)
buyer = widgets.Dropdown(
    options=vat_data_inputs.Buyer.unique(),
    value='AT',
    description='Buyer:',
    disabled=False,
)
vatid = widgets.Dropdown(
    options=vat_data_inputs.VAT_ID.unique(),
    value='AT',
    description='VAT ID:',
    disabled=False,
)
material_tax = widgets.Dropdown(
    options=vat_data_inputs.Material_Tax.unique(),
    value=1,
    description='Material_tax:',
    disabled=False,
)

In [37]:
display(Markdown('**Select observation to inspect with LIME (make sure, the combination exists)**'))
widgets.HBox([ship_from, ship_to, incoterm, buyer, vatid, material_tax])

**Select observation to inspect with LIME (make sure, the combination exists)**

In [38]:
sample_data=vat_data[vat_data['Ship_from']==ship_from.value][vat_data['Ship_to']==ship_to.value][vat_data['Incoterm']==incoterm.value][vat_data['Buyer']==buyer.value][vat_data['VAT_ID']==vatid.value][vat_data['Material_Tax']==material_tax.value].iloc[:1]
sample_data

c:\users\luvi\documents\winpython-64bit-3.6.2.0zero\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


Ship_from Ship_to Buyer  Material_Tax  Incoterm VAT_ID            Taxcode  \
0        AT      AT    AT             1         0     AT  domestic full tax   

   Taxrate  
0      0.2

### Apply Label-encoding to observation

In [39]:
rule_1_encoded_vat_data_inputs=dict()

for col in vat_data_inputs.columns:
    
    rule_1_encoded_vat_data_inputs[col] = encoders[col].transform(sample_data[col].values.astype('str'))

In [40]:
# keep original order of columns
rule_1_encoded_vat_data_inputs = pd.DataFrame(rule_1_encoded_vat_data_inputs, columns=vat_data_inputs.columns).values 

## Inspect Models with Lime

In [41]:
predict_fn = lambda x: rf.predict_proba(ohe.transform(x)) # include one hot encoding in predict function

display(Markdown('**Random Forest**'))

exp = explainer.explain_instance(rule_1_encoded_vat_data_inputs[0], predict_fn, num_samples=100,
                                     num_features=len(encoded_vat_data_inputs.columns),
                                     top_labels=1)
exp.show_in_notebook(show_table=True, show_all=False)

**Random Forest**

### Why would Incoterm be relevant?

In [42]:
pd.crosstab(vat_data.Taxcode,vat_data.Incoterm) 

Incoterm              0     1
Taxcode                      
domestic full tax  2437  2135
domestic no tax     591   181
export             1483   107
intra-EU supply    2982    84